In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def rest(df):
    game = np.array(df['G'].fillna(0))
    rested = game == 0
    df['Rest'] = rested*1
    return df

def str_to_int(s):
    if len(re.findall('\d', s)) != 0:
        if '.' in s:
            return float(s)
        return int(s[:2])
    return 0

def min_to_int(s):
    if len(re.findall('\d', s)) != 0:
        return int(s.split(':')[0])
    return 0

def plus_min(s):
    if len(re.findall('\d', s)) == 0:
        return 0
    if s[0] == '-':
        return int(s[1:]) * -1
    if s[0] == '+':
        return int(s[1:]) 
    return 0

def meaner(df, col):
    col_vals = df[col].apply(str_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def meaner_min(df, col):
    col_vals = df[col].apply(min_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def meaner_int(df, col):
    col_vals = df[col].values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def games(df, col):
    game = df[col].fillna(0)
    length = len(game)
    new_game = []
    for i in np.arange(length):
        new_game.append(max(game[:(i + 1)]))
    df[col] = new_game
    return df

def manipulate(df, inj, player):
    df = df.drop(labels=['Tm', 'GS', 'Unnamed: 5', 'Opp', 'PF', 'Unnamed: 7'], axis=1)
    df = rest(df) 
    df['Age'] = df['Age'].apply(str_to_int) 
    df['+/-'] = df['+/-'].apply(plus_min) 
    
    df = games(df, 'G')
    df = meaner_min(df, 'MP')
    df = meaner(df, 'FG')
    df = meaner(df, 'FGA')
    df = meaner(df, '3P')
    df = meaner(df, '3PA')
    df = meaner(df, 'FT')
    df = meaner(df, 'FTA')
    
    df['FG%'] = df['FG']/df['FGA']
    df['3P%'] = df['3P']/df['3PA']
    df['FT%'] = df['FT']/df['FTA']
    
    df = meaner(df, 'ORB') 
    df = meaner(df, 'DRB')
    df = meaner(df, 'TRB')
    df = meaner(df, 'AST')
    df = meaner(df, 'STL')
    df = meaner(df, 'BLK')
    df = meaner(df, 'TOV')
    df = meaner(df, 'PTS')
    df = meaner(df, 'GmSc')
    
    df = meaner_int(df, '+/-')
    
    player_i = inj[inj['Relinquised'] == player]
    player_i = player_i[['Date', 'score']]
    
    df = df.merge(player_i, how='left')
    df['score'] = df['score'].fillna(0)
    df['score'] = np.cumsum(df['score'].values)
    df = df.drop('Date', axis=1)
    return df

In [3]:
i = pd.read_csv('injury_with_score.csv')

In [ ]:
dir = ""

In [ ]:
player_names = []

In [ ]:
def generate_csv(playername):
    player_path = dir + playername + ".csv"
    df = pd.read_csv(player_path)
    new = manipulate(df, i, playername)
    player_save = dir + 'processed/' + playername.replace(" ", "_") + "_processed.csv"
    new.to_csv(player_save, index=False)

In [4]:
andre = pd.read_csv('andre_iguodala.csv')
dray = pd.read_csv('draymond_green.csv')
clark = pd.read_csv('ian_clark.csv')
kd = pd.read_csv('kevin_durant.csv')
klay = pd.read_csv('klay_thompson.csv')
barnes = pd.read_csv('matt_barnes.csv')
mccaw = pd.read_csv('patrick_mccaw.csv')
livingston = pd.read_csv('shaun_livingston.csv')
steph = pd.read_csv('stephen_curry.csv')
zaza = pd.read_csv('zaza_pachulia.csv')

FileNotFoundError: [Errno 2] File b'andre_iguodala.csv' does not exist: b'andre_iguodala.csv'

In [9]:
nandre = manipulate(andre, i, 'Andre Iguodala')
ndray = manipulate(dray, i, 'Draymond Green')
nclark = manipulate(clark, i, 'Ian Clark')
nkd = manipulate(kd, i, 'Kevin Durant')
nklay = manipulate(klay, i, 'Klay Thompson')
nbarnes = manipulate(barnes, i, 'Matt Barnes')
nmccaw = manipulate(mccaw, i, 'Patrick McCaw')
nlivingston = manipulate(livingston, i, 'Shaun Livingston')
nsteph = manipulate(steph, i, 'Stephen Curry')
nzaza = manipulate(zaza, i, 'Zaza Pachulia')